In [ ]:
# example_srs_sim.py

import numpy as np
import tensorflow as tf
from sionna.nr import CarrierConfig
from sionna.nr.srs_config import SRSConfig
from sionna.nr.srs_pilot_pattern import SRSPilotPattern
from sionna.ofdm import ResourceGrid

def main():
    # 1) Create a CarrierConfig
    carrier = CarrierConfig()
    carrier.n_size_grid = 24  # 24 RB
    carrier.subcarrier_spacing = 15
    carrier.cyclic_prefix = "normal"
    carrier.slot_number = 10
    carrier.frame_number = 0

    # 2) Create an SRSConfig for each transmitter
    srs1 = SRSConfig(carrier_config=carrier)
    srs1.num_srs_ports = 2
    srs1.symbol_start = 10
    srs1.num_srs_symbols = 2
    srs1.c_srs = 5
    srs1.b_srs = 0
    srs1.k_tc = 2
    srs1.srs_period = [2,1]  # e.g. period=2 slots, offset=1
    srs1.n_srs_id = 123

    srs2 = SRSConfig(carrier_config=carrier)
    srs2.num_srs_ports = 1
    srs2.symbol_start = 11
    srs2.num_srs_symbols = 1
    srs2.c_srs = 3
    srs2.b_srs = 1
    srs2.srs_period = 'on'   # always on

    # 3) Build a pilot pattern for them
    pilot_pattern = SRSPilotPattern([srs1, srs2])

    print("Pilot pattern created. shape of mask=", pilot_pattern.mask.shape)
    print("Pilot pattern pilots shape=", pilot_pattern.pilots.shape)

    # 4) Create a ResourceGrid for a single slot with multiple "transmitters"
    #    Suppose we want to place SRS as "pilots" in that ResourceGrid.
    num_tx = pilot_pattern.mask.shape[0]  # =2 in this example
    num_streams_per_tx = pilot_pattern.mask.shape[1] # e.g. 2 for srs1, 1 for srs2

    rg = ResourceGrid(
        num_ofdm_symbols=carrier.num_symbols_per_slot,
        fft_size=carrier.n_size_grid*12,
        num_tx=num_tx,
        num_streams_per_tx=num_streams_per_tx,
        pilot_pattern=pilot_pattern,
        cyclic_prefix_length=carrier.cyclic_prefix_length
    )

    # 5) Place the SRS signals into that resource grid
    # Sionna’s built-in pipeline will automatically map pilot_pattern
    # if you do the standard calls to build_tf_signal. For a custom approach:
    x_tx = rg.apply_pilot_pattern(None)
    print("Resource grid shape after applying pilot pattern:", x_tx.shape)

if __name__ == "__main__":
    main()
